In [ ]:
!pip install scikit-surprise

# Algorithm User-Items Based

In [36]:
import pandas as pd

# Updated dataset
# data = pd.DataFrame({
#     'user_id': [1, 1, 1, 2, 2, 2, 3, 3, 3, 3,3, 4, 4, 4],
#     'post_id': [2, 3, 4, 2, 5, 3, 2, 4, 5, 6,7, 2, 4, 9]
# })
# # Input data
# input_data = pd.DataFrame({
#     'user_id': [5, 5, 5,7,7,7],
#     'post_id': [2, 4, 5,4,9,10]
# })
data = pd.read_csv('data.csv')
input_data = pd.read_csv('input_data.csv')

In [37]:
def calculate_similarity(input_data, data):
    similar_users = {}
    user_posts = data.groupby('user_id')['post_id'].apply(set).to_dict()

    for input_user_id in input_data['user_id'].unique():
        input_user_posts = set(input_data[input_data['user_id'] == input_user_id]['post_id'])
        similar_users[input_user_id] = []

        for user_id, posts in user_posts.items():
            common_posts = input_user_posts.intersection(posts)
            if common_posts:
                similarity_score = len(common_posts) / len(input_user_posts.union(posts))
                similar_users[input_user_id].append((user_id, similarity_score))

    return similar_users

def find_top_similar_user(similar_users, top_n=1):
    top_similar_users = {}

    for input_user_id, user_similarities in similar_users.items():
        user_similarities.sort(key=lambda x: x[1], reverse=True)
        top_similar_users[input_user_id] = user_similarities[:top_n]

    return top_similar_users

def recommend_posts(input_data, similar_users, data):
    recommendations = {}

    for input_user_id, similar_user_list in similar_users.items():
        recommendations[input_user_id] = []

        for user_id, similarity_score in similar_user_list:
            user_posts = set(data[data['user_id'] == user_id]['post_id'])
            input_user_posts = set(input_data[input_data['user_id'] == input_user_id]['post_id'])
            recommended_posts = user_posts - input_user_posts
            recommendations[input_user_id].extend(recommended_posts)

    return recommendations


In [42]:
similarities = calculate_similarity(input_data, data)
top_similar_users = find_top_similar_user(similarities, top_n=1)
recommendations = recommend_posts(input_data, top_similar_users, data)

print("------------------Top similar users-----------------------\n")
for input_user_id, similar_user_list in top_similar_users.items():
    print(f"user_id {input_user_id} has top similar to:")
    for user_id, similarity_score in similar_user_list:
        print(f"  user_id {user_id} with similarity score: {similarity_score},\n")
# Additional print statements
print("------------------Matching similar personalize-----------------------")
for input_user_id, similar_user_list in top_similar_users.items():
    for user_id, similarity_score in similar_user_list:
        print(f"Input user_id {input_user_id} has post_ids: {set(input_data[input_data['user_id'] == input_user_id]['post_id'])}")
        print(f"Matching user_id {user_id} has post_ids: {set(data[data['user_id'] == user_id]['post_id'])}")
        print("\n")
print("Algorithm User Based ------>  Recommendations personalize:\n")
recommended_posts_listed = []

for input_user_id, recommended_posts in recommendations.items():
    print(f"user_id {input_user_id} should recommend the following posts:")
    print(recommended_posts)
    recommended_posts_listed.extend(recommended_posts)

print("\nRecommended posts All listed:", recommended_posts_listed)
    




------------------Top similar users-----------------------

user_id 168 has top similar to:
  user_id 18 with similarity score: 0.6,

user_id 169 has top similar to:
  user_id 20 with similarity score: 0.4,

------------------Matching similar personalize-----------------------
Input user_id 168 has post_ids: {2523, 2468, 2532}
Matching user_id 18 has post_ids: {2562, 2468, 2532, 2540, 2523}


Input user_id 169 has post_ids: {1024, 1040}
Matching user_id 20 has post_ids: {1024, 1040, 1073, 1051, 1053}


Algorithm User Based ------>  Recommendations personalize:

user_id 168 should recommend the following posts:
[2562, 2540]
user_id 169 should recommend the following posts:
[1073, 1051, 1053]

Recommended posts All listed: [2562, 2540, 1073, 1051, 1053]


In [40]:


# Filter the data DataFrame to include only recommended posts
recommended_data = data[data['post_id'].isin(recommended_posts_listed)]

# Print the filtered DataFrame
recommended_data


,user_id,post_id,Article,Author,Categories,Views
38,18,2540,AI Revolutionizing Cambodia’s Banking and Fina...,Molika Meas,438,Finance & Banking
39,18,2562,MoU to Promote Climate Resilience,Sreypich Mao,41,Finance & Banking
47,20,1051,Telcotech Signs Deal with Kampus to Expand Dat...,staff writers Kiripost,367,Tech
48,20,1053,National Domain Name Rolled Out,Mengheng Seng,110,Tech
49,20,1073,Cambodian Academics Debate Chatbot Pros and Cons,Hongseng Rov,529,Tech


# Algorithm: Items-Items Based

In [ ]:
import pandas as pd

# Updated dataset
data = pd.DataFrame({
    'user_id': [1, 1, 1, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4],
    'post_id': [2, 3, 4, 2, 6, 3, 2, 4, 5, 6, 2, 4, 9]
})
# Input data
input_data = pd.DataFrame({
    'user_id': [5, 5, 5],
    'post_id': [2, 4, 5]
})

In [12]:
# # Sample input data
# input_data = pd.DataFrame({'user_id': [3, 3], 'post_id': [3, 4]})
# data = pd.DataFrame({'user_id': [1, 1, 2, 2], 'post_id': [1, 3, 1, 3]})
def calculate_similarity(input_data, data):
    similar_users = {}
    user_posts = data.groupby('user_id')['post_id'].apply(set).to_dict()

    for input_user_id in input_data['user_id'].unique():
        input_user_posts = set(input_data[input_data['user_id'] == input_user_id]['post_id'])
        similar_users[input_user_id] = []

        for user_id, posts in user_posts.items():
            common_posts = input_user_posts.intersection(posts)
            if common_posts:
                similarity_score = len(common_posts) / len(input_user_posts.union(posts))
                similar_users[input_user_id].append((user_id, similarity_score))

    return similar_users

def find_top_similar_user(similar_users, top_n=1):
    top_similar_users = {}

    for input_user_id, user_similarities in similar_users.items():
        user_similarities.sort(key=lambda x: x[1], reverse=True)
        top_similar_users[input_user_id] = user_similarities[:top_n]

    return top_similar_users

def recommend_posts(input_data, similar_users, data):
    recommendations = {}

    for input_user_id, similar_user_list in similar_users.items():
        recommendations[input_user_id] = []

        for user_id, _ in similar_user_list:
            user_posts = set(data[data['user_id'] == user_id]['post_id'])
            input_user_posts = set(input_data[input_data['user_id'] == input_user_id]['post_id'])
            recommended_posts = user_posts - input_user_posts
            recommendations[input_user_id].extend(recommended_posts)

    return recommendations

similarities = calculate_similarity(input_data, data)
top_similar_users = find_top_similar_user(similarities, top_n=1)
recommendations = recommend_posts(input_data, top_similar_users, data)

print("------------------Top similar users-----------------------\n")
for input_user_id, similar_user_list in top_similar_users.items():
    print(f"user_id {input_user_id} has top similar to:")
    for user_id, similarity_score in similar_user_list:
        print(f"  user_id {user_id} with similarity score: {similarity_score}")
print("------------------Matching similar personalize-----------------------")
for input_user_id, similar_user_list in top_similar_users.items():
    for user_id, similarity_score in similar_user_list:
        print(f"Input user_id {input_user_id} has post_ids: \n{set(input_data[input_data['user_id'] == input_user_id]['post_id'])}")
        print(f"user_id {user_id} has post_ids: \n{set(data[data['user_id'] == user_id]['post_id'])}")
        print("\n")
print("Algorithm Item Based ------>  Recommendations personalize:\n")
for input_user_id, recommended_posts in recommendations.items():
    print(f"user_id {input_user_id} should recommend the following posts:")
    print(recommended_posts)


------------------Top similar users-----------------------

user_id 168 has top similar to:
  user_id 18 with similarity score: 0.6
user_id 169 has top similar to:
  user_id 20 with similarity score: 0.4
------------------Matching similar personalize-----------------------
Input user_id 168 has post_ids: 
{2523, 2468, 2532}
user_id 18 has post_ids: 
{2562, 2468, 2532, 2540, 2523}


Input user_id 169 has post_ids: 
{1024, 1040}
user_id 20 has post_ids: 
{1024, 1040, 1073, 1051, 1053}


Algorithm Item Based ------>  Recommendations personalize:

user_id 168 should recommend the following posts:
[2562, 2540]
user_id 169 should recommend the following posts:
[1073, 1051, 1053]


# Items-based

In [35]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import spacy

# Load data from CSV files
data = pd.read_csv('data.csv')
input_data = pd.read_csv('input_data.csv')

# Load English language model for spaCy
nlp = spacy.load('en_core_web_sm')

def preprocess_text(text):
    # Tokenize, lemmatize, and remove stop words
    if isinstance(text, str):
        doc = nlp(text)
        tokens = [token.lemma_ for token in doc if not token.is_stop]
        processed_text = ' '.join(tokens)
        return processed_text
    else:
        return ""

# Apply preprocessing to relevant columns
data['Article_processed'] = data['Article'].apply(preprocess_text)
data['Author_processed'] = data['Author'].apply(preprocess_text)
data['Categories_processed'] = data['Categories'].apply(preprocess_text)

def calculate_similarity(data):
    # Check if data is not empty
    if data.empty:
        raise ValueError("Input data is empty")

    # Prepare feature vectors
    tfidf_vectorizer = TfidfVectorizer()
    item_features = tfidf_vectorizer.fit_transform(data['Article_processed'] + ' ' + data['Author_processed'] + ' ' + data['Categories_processed'])

    # Compute cosine similarity
    item_similarity_matrix = cosine_similarity(item_features, item_features)

    return item_similarity_matrix

def find_similar_items(target_item_index, item_similarity_matrix, data, top_n=5):
    # Check if target item index is valid
    if target_item_index >= len(data):
        raise ValueError("Invalid target item index")

    # Retrieve attributes of target item
    target_item_attributes = data.iloc[target_item_index]

    # Calculate similarity
    similarity_scores = item_similarity_matrix[target_item_index]

    # Sort similar items
    similar_item_indices = similarity_scores.argsort()[::-1][1:top_n+1]

    # Output similar items
    similar_items = data.iloc[similar_item_indices]

    return similar_items

# Calculate item similarity matrix
item_similarity_matrix = calculate_similarity(data)

# Find index of target item
target_item_index = data[(data['user_id'] == input_data['user_id'].values[0]) & (data['post_id'] == input_data['post_id'].values[0])].index
if not target_item_index.empty:
    target_item_index = target_item_index[0]
    # Find similar items
    similar_items = find_similar_items(target_item_index, item_similarity_matrix, data)

    print("Recommended similar items:")
    print(similar_items)
else:
    print("No item found for the given user and post ID.")


No item found for the given user and post ID.


# Personalize recommandations Systems Collaboration Filtering

In [18]:
# import pandas as pd

# # Updated dataset
# data = pd.DataFrame({
#     'user_id': [1, 1, 1, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4],
#     'post_id': [2, 4, 6, 2, 4, 3, 2, 4, 5, 6, 2, 4, 9]
# })
# # Input data
# input_data = pd.DataFrame({
#     'user_id': [5, 5, 5, 7, 7, 7],
#     'post_id': [2, 4, 5, 4, 9, 10]
# })

In [13]:
from collections import defaultdict

def calculate_similarity(input_data, data):
    similar_users = defaultdict(list)
    similar_items = defaultdict(list)
    user_posts = data.groupby('user_id')['post_id'].apply(set).to_dict()

    for input_user_id in input_data['user_id'].unique():
        input_user_posts = set(input_data[input_data['user_id'] == input_user_id]['post_id'])

        for user_id, posts in user_posts.items():
            common_posts = input_user_posts.intersection(posts)
            if common_posts:
                similarity_score = len(common_posts) / len(input_user_posts.union(posts))
                similar_users[input_user_id].append((user_id, similarity_score))
                for item in common_posts:
                    similar_items[item].append((user_id, similarity_score))

    return similar_users, similar_items

def find_top_similar_user(similar_users, top_n=1):
    top_similar_users = {}

    for input_user_id, user_similarities in similar_users.items():
        user_similarities.sort(key=lambda x: x[1], reverse=True)
        top_similar_users[input_user_id] = user_similarities[:top_n]

    return top_similar_users

def recommend_posts(input_data, similar_users, similar_items, data):
    recommendations = defaultdict(list)

    # Merge similar_users and similar_items into a single loop
    for input_user_id, similar_list in similar_users.items():
        similar_items_list = similar_items[input_user_id]

        recommended_posts = set()  # Set to store unique recommended post_ids

        for user_id, _ in similar_list:
            user_posts = set(data[data['user_id'] == user_id]['post_id'])
            input_user_posts = set(input_data[input_data['user_id'] == input_user_id]['post_id'])
            recommended_posts.update(user_posts - input_user_posts)

        for user_id, _ in similar_items_list:
            user_posts = set(data[data['user_id'] == user_id]['post_id'])
            input_user_posts = set(input_data[input_data['user_id'] == input_user_id]['post_id'])
            recommended_posts.update(user_posts - input_user_posts)

        recommendations[input_user_id].extend(recommended_posts)

    return recommendations




similar_users, similar_items = calculate_similarity(input_data, data)
top_similar_users = find_top_similar_user(similar_users, top_n=1)
recommendations = recommend_posts(input_data, top_similar_users, similar_items, data)

print("Top similar users:")
for input_user_id, similar_user_list in top_similar_users.items():
    print(f"user_id {input_user_id} has top similar to:")
    for user_id, similarity_score in similar_user_list:
        print(f"  user_id {user_id} with similarity score: {similarity_score}")
print("------------------Matching similar personalize-----------------------")
for input_user_id, similar_user_list in top_similar_users.items():
    for user_id, similarity_score in similar_user_list:
        print(f"Input user_id {input_user_id} has post_ids: \n{set(input_data[input_data['user_id'] == input_user_id]['post_id'])}")
        print(f"user_id {user_id} has post_ids: \n{set(data[data['user_id'] == user_id]['post_id'])}")
        print("\n")
print("\nRecommendations:")
for input_user_id, recommended_posts in recommendations.items():
    print(f"user_id {input_user_id} should recommend the following posts:")
    print(recommended_posts)


Top similar users:
user_id 168 has top similar to:
  user_id 18 with similarity score: 0.6
user_id 169 has top similar to:
  user_id 20 with similarity score: 0.4
------------------Matching similar personalize-----------------------
Input user_id 168 has post_ids: 
{2523, 2468, 2532}
user_id 18 has post_ids: 
{2562, 2468, 2532, 2540, 2523}


Input user_id 169 has post_ids: 
{1024, 1040}
user_id 20 has post_ids: 
{1024, 1040, 1073, 1051, 1053}



Recommendations:
user_id 168 should recommend the following posts:
[2562, 2540]
user_id 169 should recommend the following posts:
[1073, 1051, 1053]
